In [1]:
import torch
from tqdm import tqdm
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../GGH')

from GGH.data_ops import DataOperator
from GGH.selection_algorithms import AlgoModulators
from GGH.models import initialize_model, Autoencoder
from GGH.train_val_loop import TrainValidationManager
from GGH.inspector import Inspector, plot_histogram, visualize_train_val_error, selection_histograms, create_heatmap, cluster_and_visualize, \
                            get_gradarrays_n_labels, prep_tsne_min_max_ranges, sep_grads_by_labels
from GGH.imputation_methods import Imputer
import warnings
warnings.filterwarnings('ignore')

def full_experiment(use_info, DO, INSPECT, batch_size, hidden_size, output_size, num_epochs, rand_state, results_path, dropout = 0.05, lr = 0.004,
                    nu = 0.1, normalize_grads_contx = False, use_context = True, final_analysis = True):
       
    AM = AlgoModulators(DO, lr = lr, nu = nu)
    dataloader = DO.prep_dataloader(use_info, batch_size)

    model = initialize_model(DO, dataloader, hidden_size, rand_state, dropout = dropout) 

    TVM = TrainValidationManager(use_info, num_epochs, dataloader, batch_size, rand_state, results_path, final_analysis = final_analysis)
    TVM.train_model(DO, AM, model, final_analysis = final_analysis)

    INSPECT.save_train_val_logs(DO, AM, TVM, model, final_analysis = final_analysis)
    
    return DO, TVM, model

def multi_experiments(total_runs, use_info, num_epochs, data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc,
                      INSPECT, batch_size, hidden_size, output_size, results_path, hyperparameters, final_analysis = True):
    
    progress_bar = tqdm(total=total_runs)
    for r_state in range(2000): #
        DO = DataOperator(data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc, r_state, device = "cpu")
        if not DO.lack_partial_coverage:
            full_experiment(use_info, DO, INSPECT, batch_size, hidden_size, output_size, num_epochs, r_state, results_path,
                            hyperparameters["dropout"]["value"], hyperparameters["lr"]["value"], hyperparameters["nu"]["value"],
                            hyperparameters["normalize_grads_contx"]["value"], hyperparameters["use_context"]["value"], final_analysis)
            progress_bar.update(1)
        if progress_bar.n == total_runs:
            break       
    progress_bar.close()

import sys
import io
import contextlib
def allowed_print(*args, **kwargs):
    """Print immediately to the notebook output even when stdout is redirected."""
    print(*args, **kwargs, file=_real_stdout)

2025-12-04 16:12:32.175108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 16:12:33.075109: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-12-04 16:12:33.075246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2025-12-04 16:12:33.075256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
#User requiered data and parameters
data_path = '../data/dataset_photo_pce10/data.csv'
results_path = "../saved_results/Photocell Degradation"
inpt_vars = ['P3HT', 'PTB7-Th']
target_vars = ['Degradation']
miss_vars = ['PCBM']
hypothesis = [[0.03, 0.11, 0.20, 0.32, 0.43, 0.6]]

#Percentage of simulated data with full information
partial_perc = 0.015
rand_state = 42

#Model Train Parameters
hidden_size = 32

batch_size = 100*len(hypothesis[0])
output_size = len(target_vars)

#Call data, algorithm and model classes
INSPECT = Inspector(results_path, hidden_size)

In [3]:
partial_perc = 0.03
rand_state = 42

use_info = "use hypothesis"
num_epochs = 100
DO = DataOperator(data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc, rand_state, device = "cpu")
DO, TVM, model = full_experiment(use_info, DO, INSPECT, batch_size, hidden_size, output_size, num_epochs, rand_state, 
                                 results_path, dropout = 0.05, nu = 0.1, final_analysis = False)

print(INSPECT.calculate_val_r2score(DO, TVM, model, data = "validation"))
print(INSPECT.calculate_val_r2score(DO, TVM, model, data = "test"))

selection_histograms(DO, TVM, num_epochs, rand_state, partial_perc)

 39%|███▉      | 39/100 [02:47<04:22,  4.30s/it]

KeyboardInterrupt



In [3]:
#Number of loops to ensure statistical significance
num_loops = 15
num_epochs = 200

hyperparameters = {"lr": {"value": 0.004}, "dropout": {"value": 0.05}, "nu": {"value": 0.1} ,
                   "normalize_grads_contx": {"value":False}, "use_context": {"value":True}}

In [ ]:
multi_experiments(num_loops, "use hypothesis", num_epochs, data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc, 
                  INSPECT, batch_size, hidden_size, output_size, results_path, hyperparameters)

In [8]:
%%capture
multi_experiments(num_loops, "partial info",   int(num_epochs*2.5), data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc, 
                  INSPECT, batch_size, hidden_size, output_size, results_path, hyperparameters)
multi_experiments(num_loops, "use known only", int(num_epochs*2.5), data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc,
                  INSPECT, batch_size, hidden_size, output_size, results_path, hyperparameters)
multi_experiments(num_loops, "full info",      int(num_epochs*2.5), data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc,
                  INSPECT, batch_size, hidden_size, output_size, results_path, hyperparameters)

In [5]:
#from tabpfn import TabPFNRegressor
from tabpfn import TabPFNClassifier

In [ ]:
buf = io.StringIO()
_real_stdout = sys.stdout
with contextlib.redirect_stdout(buf):
    for imput_method in ["Iterative MICE Imputer", "KNN Imputer", "Miss Forest", "Deep Regressor", "Soft Impute", 
                         "Matrix Factorization", "Hyperimpute", "TabPFN"]: #, "Midas"
        allowed_print(f"Executing benchmark for {imput_method}...")
        counter = 0
        use_info = "use imputation"
        num_epochs = 500
        for r_state in range(300):
            DO = DataOperator(data_path, inpt_vars, target_vars, miss_vars, hypothesis, partial_perc, r_state, device = "cpu")
            if not DO.lack_partial_coverage:
                counter += 1
                AM = AlgoModulators(DO, lr = 0.004)
                IMP = Imputer(DO)
                dataloader = IMP.impute_w_sel(DO, imput_method, batch_size)

                model = initialize_model(DO, dataloader, hidden_size, r_state, dropout = 0.05) 
                TVM = TrainValidationManager(use_info, num_epochs, dataloader, batch_size, r_state, results_path,
                                            imput_method = imput_method)
                TVM.train_model(DO, AM, model)
                INSPECT.save_train_val_logs(DO, AM, TVM, model, imput_method, final_analysis = True)
            if counter == 15: #15:
                break

Executing benchmark for Iterative MICE Imputer...
Executing benchmark for KNN Imputer...
Executing benchmark for Miss Forest...
Executing benchmark for Deep Regressor...
Executing benchmark for Soft Impute...
Executing benchmark for Matrix Factorization...
Executing benchmark for Hyperimpute...


/home/pneves6/.conda/envs/bee_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: DeprecationWarning: The 'sym_pos' keyword is deprecated and should be replaced by using 'assume_a = "pos"'. 'sym_pos' will be removed in SciPy 1.11.0.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/pneves6/.conda/envs/bee_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: DeprecationWarning: The 'sym_pos' keyword is deprecated and should be replaced by using 'assume_a = "pos"'. 'sym_pos' will be removed in SciPy 1.11.0.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/pneves6/.conda/envs/bee_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: DeprecationWarning: The 'sym_pos' keyword is deprecated and should be replaced by using 'assume_a = "pos"'. 'sym_pos' will be removed in SciPy 1.11.0.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/pneves6/.conda/envs/bee_py38/lib/python3.8/site-package

In [ ]:
INSPECT.create_comparison_table(final_analysis = True, use_info= "use imputation")

In [5]:
INSPECT.create_comparison_table(final_analysis = True, use_info= "use imputation")

,Method,valid_min_errors_mean,valid_min_errors_std
0,Iterative MICE Imputer,0.008932,0.001600
1,KNN Imputer,0.008831,0.001528
2,Miss Forest,0.009174,0.001501
3,Deep Regressor,0.009396,0.001246
4,Soft Impute,0.009045,0.001269
5,Matrix Factorization,0.008636,0.000813
6,Midas,0.008565,0.001596


In [14]:
df, df_notavg = INSPECT.create_test_comparison_table(data_path, inpt_vars, target_vars, miss_vars, hypothesis, 
                                                     partial_perc, batch_size, best_imput = "Matrix Factorization")

df.dropna(inplace = True)
df[["Method", "avg_r2_score", "avg_mse", "avg_mae"]]

,Method,avg_r2_score,avg_mse,avg_mae
0,full info,0.878819,0.001390,0.025006
1,partial info,0.129691,0.009974,0.072821
2,use hypothesis,0.229352,0.008651,0.068198
4,use known only,0.078990,0.010703,0.076099


In [6]:
df, df_notavg = INSPECT.create_test_comparison_table(data_path, inpt_vars, target_vars, miss_vars, hypothesis, 
                                                     partial_perc, batch_size, best_imput = "Midas")

df.dropna(inplace = True)
df[["Method","avg_r2_score","avg_mse","avg_mae"]]

,Method,avg_r2_score,avg_mse,avg_mae
0,use hypothesis,0.325064,0.008740,0.068323
1,use known only,0.143105,0.011313,0.077256
2,partial info,0.137756,0.011097,0.078377
3,full info,0.879808,0.001529,0.026339
4,use imputation,0.178574,0.010541,0.075807
